In [1]:
#import pandas
import os
import pandas as pd 

**Step 1 load data**

In [2]:
#load data
aisles = pd.read_csv('datasets_4931_7487_aisles.csv') 
departments = pd.read_csv('datasets_4931_7487_departments.csv')
products_prior = pd.read_csv('order_products__prior.csv')
products_train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

Aisles df key id : aisle_id

In [3]:
#inspect data 
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


departments key id: department_id 

In [4]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


products key id: order_id, product_id 
gives order added to cart, and if reordered or not 

In [5]:
products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


products key id: order_id, product_id 
gives order added to cart, and if reordered or not 

In [6]:
products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


key id: order id, eval_set : gives which set the order is from, order_dow: day of week

In [7]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


key id : product id 

In [8]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


**Step 2 merge dataframes**

In [9]:

product_info = products.merge(aisles, on = 'aisle_id',  how='inner')
product_info.head()

,product_id,product_name,aisle_id,department_id,aisle
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes
2,102,Danish Butter Cookies,61,19,cookies cakes
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes


In [10]:
product_info = product_info.merge(departments, on = 'department_id', how='inner')
product_info.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


In [11]:
#merge orders and products_prior : 

prior_orders = orders.merge(products_prior, on = 'order_id', how = 'inner')
prior_orders = prior_orders.merge(product_info, on = 'product_id', how = 'inner')
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,soft drinks,beverages
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,soft drinks,beverages
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,soft drinks,beverages
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,soft drinks,beverages
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,soft drinks,beverages


In [12]:
user = prior_orders.groupby('user_id')['order_number'].max().to_frame('u_total_orders')
user.head()

,u_total_orders
user_id,
1,10
2,14
3,12
4,5
5,4


In [13]:
user_avg_reorder = prior_orders.groupby('user_id')['reordered'].mean().to_frame('avg_reorder')

user = user.merge(user_avg_reorder, on = 'user_id', how = 'inner')
user.head()

,u_total_orders,avg_reorder
user_id,,
1,10,0.694915
2,14,0.476923
3,12,0.625000
4,5,0.055556
5,4,0.378378


In [14]:
product = prior_orders.groupby('product_id')['order_id'].count().to_frame('p_total_purchases')
product.head()

,p_total_purchases
product_id,
1,1852
2,90
3,277
4,329
5,15


In [15]:
product_reorder = prior_orders.groupby('product_id')['reordered'].sum().to_frame('p_total_reorders')
product = product.merge(product_reorder, on = 'product_id', how= 'inner')
product['percent_reorder'] = (product['p_total_reorders']/product['p_total_purchases'])
product.head()

,p_total_purchases,p_total_reorders,percent_reorder
product_id,,,
1,1852,1136,0.613391
2,90,12,0.133333
3,277,203,0.732852
4,329,147,0.446809
5,15,9,0.600000


In [16]:
product = pd.merge(product, product_info, on='product_id', how ='inner')
product.head()

,product_id,p_total_purchases,p_total_reorders,percent_reorder,product_name,aisle_id,department_id,aisle,department
0,1,1852,1136,0.613391,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,90,12,0.133333,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,277,203,0.732852,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,329,147,0.446809,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,15,9,0.600000,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


In [17]:
product_sorted = product.sort_values(by= 'p_total_purchases', ascending = False)
product_sorted.head(15)

,product_id,p_total_purchases,p_total_reorders,percent_reorder,product_name,aisle_id,department_id,aisle,department
24848,24852,472565,398609,0.843501,Banana,24,4,fresh fruits,produce
13172,13176,379450,315913,0.832555,Bag of Organic Bananas,24,4,fresh fruits,produce
21133,21137,264683,205845,0.777704,Organic Strawberries,24,4,fresh fruits,produce
21899,21903,241921,186884,0.772500,Organic Baby Spinach,123,4,packaged vegetables fruits,produce
47198,47209,213584,170131,0.796553,Organic Hass Avocado,24,4,fresh fruits,produce
47755,47766,176815,134044,0.758103,Organic Avocado,24,4,fresh fruits,produce
47615,47626,152657,106255,0.696038,Large Lemon,24,4,fresh fruits,produce
16793,16797,142951,99802,0.698155,Strawberries,24,4,fresh fruits,produce
26204,26209,140627,95768,0.681007,Limes,24,4,fresh fruits,produce
27839,27845,137905,114510,0.830354,Organic Whole Milk,84,16,milk,dairy eggs


In [18]:
# merge orders and products train:

train_orders = orders.merge(products_train, on = 'order_id', how = 'inner')
train_orders = train_orders.merge(product_info, on = 'product_id', how = 'inner')
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1187899,1,train,11,4,8,14.0,196,1,1,Soda,77,7,soft drinks,beverages
1,2757217,67,train,25,0,11,5.0,196,1,1,Soda,77,7,soft drinks,beverages
2,632715,676,train,12,0,13,26.0,196,2,1,Soda,77,7,soft drinks,beverages
3,1167274,760,train,5,4,10,8.0,196,2,1,Soda,77,7,soft drinks,beverages
4,3347074,804,train,16,3,21,5.0,196,4,0,Soda,77,7,soft drinks,beverages


**Step 3 Data Definition**

In [19]:
prior_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
 7   product_id              int64  
 8   add_to_cart_order       int64  
 9   reordered               int64  
 10  product_name            object 
 11  aisle_id                int64  
 12  department_id           int64  
 13  aisle                   object 
 14  department              object 
dtypes: float64(1), int64(10), object(4)
memory usage: 3.9+ GB


In [20]:
train_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 0 to 1384616
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   order_id                1384617 non-null  int64  
 1   user_id                 1384617 non-null  int64  
 2   eval_set                1384617 non-null  object 
 3   order_number            1384617 non-null  int64  
 4   order_dow               1384617 non-null  int64  
 5   order_hour_of_day       1384617 non-null  int64  
 6   days_since_prior_order  1384617 non-null  float64
 7   product_id              1384617 non-null  int64  
 8   add_to_cart_order       1384617 non-null  int64  
 9   reordered               1384617 non-null  int64  
 10  product_name            1384617 non-null  object 
 11  aisle_id                1384617 non-null  int64  
 12  department_id           1384617 non-null  int64  
 13  aisle                   1384617 non-null  object 
 14  de

In [21]:
#change type to category
train_orders['aisle'] = train_orders['aisle'].astype('category')
train_orders['department'] = train_orders['department'].astype('category')
train_orders['eval_set'] = train_orders['eval_set'].astype('category')
train_orders['product_name'] = train_orders['product_name'].astype('category')
prior_orders['aisle'] = prior_orders['aisle'].astype('category')
prior_orders['department'] = prior_orders['department'].astype('category')
prior_orders['eval_set'] = prior_orders['eval_set'].astype('category')
prior_orders['product_name'] = prior_orders['product_name'].astype('category')

In [22]:
train_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 0 to 1384616
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype   
---  ------                  --------------    -----   
 0   order_id                1384617 non-null  int64   
 1   user_id                 1384617 non-null  int64   
 2   eval_set                1384617 non-null  category
 3   order_number            1384617 non-null  int64   
 4   order_dow               1384617 non-null  int64   
 5   order_hour_of_day       1384617 non-null  int64   
 6   days_since_prior_order  1384617 non-null  float64 
 7   product_id              1384617 non-null  int64   
 8   add_to_cart_order       1384617 non-null  int64   
 9   reordered               1384617 non-null  int64   
 10  product_name            1384617 non-null  category
 11  aisle_id                1384617 non-null  int64   
 12  department_id           1384617 non-null  int64   
 13  aisle                   1384617 non-null  

In [23]:
prior_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 15 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   eval_set                category
 3   order_number            int64   
 4   order_dow               int64   
 5   order_hour_of_day       int64   
 6   days_since_prior_order  float64 
 7   product_id              int64   
 8   add_to_cart_order       int64   
 9   reordered               int64   
 10  product_name            category
 11  aisle_id                int64   
 12  department_id           int64   
 13  aisle                   category
 14  department              category
dtypes: category(4), float64(1), int64(10)
memory usage: 3.1 GB


In [24]:
prior_orders.agg([min, max]).T

,min,max
order_id,2.0,3421083.0
user_id,1.0,206209.0
order_number,1.0,99.0
order_dow,0.0,6.0
order_hour_of_day,0.0,23.0
days_since_prior_order,0.0,30.0
product_id,1.0,49688.0
add_to_cart_order,1.0,145.0
reordered,0.0,1.0
aisle_id,1.0,134.0


In [25]:
train_orders.agg([min,max]).T

,min,max
order_id,1.0,3421070.0
user_id,1.0,206209.0
order_number,4.0,100.0
order_dow,0.0,6.0
order_hour_of_day,0.0,23.0
days_since_prior_order,0.0,30.0
product_id,1.0,49688.0
add_to_cart_order,1.0,80.0
reordered,0.0,1.0
aisle_id,1.0,134.0


In [26]:
user.agg([min,max]).T

,min,max
u_total_orders,3.0,99.000000
avg_reorder,0.0,0.989529


In [27]:
prior_orders.agg([min, max]).T

,min,max
order_id,2.0,3421083.0
user_id,1.0,206209.0
order_number,1.0,99.0
order_dow,0.0,6.0
order_hour_of_day,0.0,23.0
days_since_prior_order,0.0,30.0
product_id,1.0,49688.0
add_to_cart_order,1.0,145.0
reordered,0.0,1.0
aisle_id,1.0,134.0


In [28]:
prior_orders.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.035642e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07
mean,1.710749e+06,1.029372e+05,1.714205e+01,2.738818e+00,1.342498e+01,1.110407e+01,2.557634e+04,8.351076e+00,5.896975e-01,7.121430e+01,9.921906e+00
std,9.873007e+05,5.946648e+04,1.753504e+01,2.090049e+00,4.246365e+00,8.778914e+00,1.409669e+04,7.126671e+00,4.918886e-01,3.820302e+01,6.281156e+00
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,8.559430e+05,5.142100e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.353000e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00
50%,1.711048e+06,1.026110e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,2.525600e+04,6.000000e+00,1.000000e+00,8.300000e+01,9.000000e+00
75%,2.565514e+06,1.543910e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,3.793500e+04,1.100000e+01,1.000000e+00,1.070000e+02,1.600000e+01
max,3.421083e+06,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.450000e+02,1.000000e+00,1.340000e+02,2.100000e+01


In [29]:
train_orders.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id
count,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06
mean,1.706298e+06,1.031128e+05,1.709141e+01,2.701392e+00,1.357759e+01,1.706613e+01,2.555624e+04,8.758044e+00,5.985944e-01,7.130423e+01,9.839777e+00
std,9.897326e+05,5.948715e+04,1.661404e+01,2.167646e+00,4.238458e+00,1.042642e+01,1.412127e+04,7.423936e+00,4.901829e-01,3.810409e+01,6.292830e+00
min,1.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,8.433700e+05,5.173200e+04,6.000000e+00,1.000000e+00,1.000000e+01,7.000000e+00,1.338000e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00
50%,1.701880e+06,1.029330e+05,1.100000e+01,3.000000e+00,1.400000e+01,1.500000e+01,2.529800e+04,7.000000e+00,1.000000e+00,8.300000e+01,8.000000e+00
75%,2.568023e+06,1.549590e+05,2.100000e+01,5.000000e+00,1.700000e+01,3.000000e+01,3.794000e+04,1.200000e+01,1.000000e+00,1.070000e+02,1.600000e+01
max,3.421070e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,8.000000e+01,1.000000e+00,1.340000e+02,2.100000e+01


**Step 4 : Data Cleaning**

In [30]:
prior_orders.isnull().values.sum() 

2078068

In [31]:
nas=pd.DataFrame(prior_orders.isnull().sum().sort_values(ascending=False)/len(prior_orders),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
days_since_prior_order,0.06407


In [32]:
nas_t=pd.DataFrame(train_orders.isnull().sum().sort_values(ascending=False)/len(train_orders),columns = ['percent'])
pos_t = nas_t['percent'] > 0
nas_t[pos_t]

,percent
